## 3.1 Data transferred and ingested in butler and access to data

Get the same ras as we did on the summit

In [16]:
from lsst.daf.butler import Butler
from lsst.daf.butler.registry import Registry
import sqlalchemy

repo = '/repo/embargo'
# raw all colleciton contains all images 
collections=['LATISS/raw/all']
butler = Butler(repo)
cols = butler.registry.queryCollections("*LATISS*raw*")
for c in cols :
    print(c)


butler = Butler(repo, collections=collections)
registry = butler.registry



LATISS/raw/all


In [32]:
dataId={'day_obs':20230509}

dt='raw'
where="exposure.day_obs = 20230505"
datasetRefs = list(registry.queryDatasets(datasetType=dt,where=where))

print(f"There are {len(datasetRefs)} {dt} in collection: {collections} in: {repo} for {where}")

count = 0
for ref in datasetRefs[0], datasetRefs[-1]:
    count = count + 1
    print( ref.dataId['detector'], ref.dataId['exposure'])



There are 138 raw in collection: ['LATISS/raw/all'] in: /repo/embargo for exposure.day_obs = 20230505
0 2023050500001
0 2023050500138


In [33]:
#Make a map of thoose for look up below .. 
dslookup = {}
for ref in datasetRefs:
    dslookup[ref.dataId['exposure']] = ref

### latency ..
"Measure the delays between completion of readout for each image and the ingestion time recorded in the Butler Registry."
The end of readou is the exposure.timespan.end.
Will have to lookup the ingest time usinf a sql query sine is not exposed directly in butler.

In [37]:
from astropy import time

res = butler.registry.queryDimensionRecords('exposure', where=where)
print (f"Got Metadata for {res.count()} exposures")
times = []
min = 100
max = 0
for count,dimensionRecord in enumerate(res):
    datasetRef = dslookup[dimensionRecord.id]
    uuid = datasetRef.id
    # the ID in the dimesnion record is the exposure ID (dimensionRecord.id)
    # the ID in the oga.dataset is the UUID from the datasetRef 
    # hence the lookup map created in teh cell above is used here to lookup the UUID
    
    q_string = f"SELECT ingest_date FROM oga.dataset WHERE id='{uuid}'"
    with registry._db.query(sqlalchemy.text(q_string)) as q:
        intime = time.Time(q.all()[0][0])
        
    diff = (intime - dimensionRecord.timespan.end ).to('minute')
    if diff.value < min:
        min = diff.value
    if diff.value > max:
        max = diff.value 
    #print(r.id,r.timespan.end, intime, diff )
    times.append(diff.value)
    
import numpy as np
print (f"The min time was {min} max time was {max} minutes")
print (f"Time between shutter close and ingest mean:{np.mean(times)}, std:{np.std(times)}, var:{np.var(times)} minutes")

Got Metadata for 138 exposures
The min time was 2.696276116666727 max time was 10.21479046666661 minutes
Time between shutter close and ingest mean:5.4056913442029195, std:1.9467945347980924, var:3.7900089607197214 minutes
